<a href="https://colab.research.google.com/github/Nahla-yasmine/Arabic-News-Scraper/blob/main/FATABAYANO_webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install selenium


In [9]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv


In [10]:

# Define column names
COLUMN_NAMES = ['Title', 'Label', 'Category', 'Content', 'Date', 'URL']

# Function to get article title from URL
def get_article_title():
    try:
        # Find and extract the title element
        title_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.w-post-elm.post_title.us_custom_d6b5cf89.entry-title.color_link_inherit')))
        title = title_element.text.strip()
        return title
    except Exception as e:
        print("An error occurred while extracting article title:", e)
        return None


In [11]:

# Function to get article info (title, content, date) from URL
def get_article_info(url):
    # Open article URL in a new tab
    driver.execute_script("window.open('" + url + "', 'new_window')")
    # Switch to the newly opened tab
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(5)  # Wait for the page to load (you can adjust the waiting time as needed)
    # Extract article title
    article_title = get_article_title()
    # Extract article content
    article_content = ""
    article_date = ""
    try:
        # Wait for the article content to load
        article_content_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.wpb_text_column.us_custom_94411672.ob-post-text')))
        paragraphs = article_content_element.find_elements(By.TAG_NAME, 'p')
        article_content = '\n'.join([p.text.strip() for p in paragraphs])
        # Extract article date
        date_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'time.w-post-elm.post_date.us_custom_0fd75781.entry-date.published')))
        article_date = date_element.get_attribute('datetime')
    except Exception as e:
        print("An error occurred while extracting article info:", e)
    finally:
        # Close the new tab
        driver.close()
        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
    return article_title, article_content, article_date


In [12]:

# Function to scrape article title, label, category, content, and URL
def scrape_article(article):
    try:
        # Extract category from article
        category_element = WebDriverWait(article, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.w-post-elm.post_taxonomy.usg_post_taxonomy_2.style_badge.color_link_inherit')))
        category_label_element = category_element.find_element(By.CSS_SELECTOR, 'span.w-btn-label')
        category = category_label_element.text.strip()
        # Define label based on category
        label = 'رأي' if category == 'رأي' else 'صحيح' if category == 'صحيح' else 'زائف'
        # Extract title from article
        title_element = article.find_element(By.CSS_SELECTOR, 'h2.w-post-elm.post_title.usg_post_title_1.has_text_color.entry-title.color_link_inherit a')
        title = title_element.get_attribute('title')
        article_url = title_element.get_attribute('href')

        # Get article info (title, content, date) from new tab
        article_title, article_content, article_date = get_article_info(article_url)

        return article_title, label, category, article_content, article_date, article_url
    except Exception as e:
        print("An error occurred while processing an article:", e)
        return None, None, None, None, None, None


In [7]:

# Function to save data to CSV file
def save_to_csv(data):
    with open('FATABAYYANO_articles_RAW_data.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row only if the file is empty
        if file.tell() == 0:
            writer.writerow(COLUMN_NAMES)
        for item in data:
            writer.writerow(item)

# Initialize the WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)


In [ ]:

try:
    # URLs to scrape
    base_url = 'https://fatabyyano.net/page/'
    page_count = 177  # Number of pages to scrape

    for page_number in range(1, page_count + 1):
        # Initialize list to store data for this page
        data = []

        url = base_url + str(page_number) + '/'
        # Open the webpage
        driver.get(url)

        # Find and extract the news articles on the current page
        news_articles = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.w-grid-item.size_1x1')))

        # Output the articles on the current page
        for article in news_articles:
            # Scrape article title, label, category, content, date, and URL
            title, label, category, article_content, article_date, article_url = scrape_article(article)
            if title is not None:
                data.append([title, label, category, article_content, article_date, article_url])

        # Save the data to CSV file after processing each page
        save_to_csv(data)

except Exception as e:
    print("An error occurred:", e)

finally:
    # Close the WebDriver
    driver.quit()


An error occurred while processing an article: Message: 
Stacktrace:
#0 0x5b0dccdc4863 <unknown>
#1 0x5b0dccaba8c6 <unknown>
#2 0x5b0dccb05618 <unknown>
#3 0x5b0dccb056d1 <unknown>
#4 0x5b0dccaf9fa6 <unknown>
#5 0x5b0dccb275cd <unknown>
#6 0x5b0dccaf9e98 <unknown>
#7 0x5b0dccb2776e <unknown>
#8 0x5b0dccb45c19 <unknown>
#9 0x5b0dccb27343 <unknown>
#10 0x5b0dccaf8593 <unknown>
#11 0x5b0dccaf8f5e <unknown>
#12 0x5b0dccd8884b <unknown>
#13 0x5b0dccd8c7a5 <unknown>
#14 0x5b0dccd76571 <unknown>
#15 0x5b0dccd8d332 <unknown>
#16 0x5b0dccd5b87f <unknown>
#17 0x5b0dccdb3728 <unknown>
#18 0x5b0dccdb38fb <unknown>
#19 0x5b0dccdc39b4 <unknown>
#20 0x7b035b1bcac3 <unknown>

An error occurred while processing an article: Message: 
Stacktrace:
#0 0x5b0dccdc4863 <unknown>
#1 0x5b0dccaba8c6 <unknown>
#2 0x5b0dccb05618 <unknown>
#3 0x5b0dccb056d1 <unknown>
#4 0x5b0dccaf9fa6 <unknown>
#5 0x5b0dccb275cd <unknown>
#6 0x5b0dccaf9e98 <unknown>
#7 0x5b0dccb2776e <unknown>
#8 0x5b0dccb45c19 <unknown>
#9 0x5b0dc